# Notebook to test Flair performance

# Setup and data cleaning

## Change logs settings

In [ ]:
import pandas as pd
import numpy as np

from datasets import disable_progress_bar
disable_progress_bar() # Disable the "Map" progress bar during the tests

## Load and clean the dataset

This dataset is not on the GitHub repository.
It's composed of work experienced fetched from LinkedIn and labelled between 0 and 4 (0 if it's not related to AI and 4 if it is)

In [ ]:
dataFrame = pd.read_pickle(r'../data/7587_corrige.pkl')
subset = dataFrame[['jobTitle', 'description', 'label']].copy()

subset.reset_index(drop=True, inplace=True)
subset.replace('', np.nan, inplace=True)
subset.dropna(inplace=True)

subset['text'] = subset['jobTitle'] + ' ' + subset['description']
subset = subset[['text','label']]
subset_label_transform = subset.copy()

subset_label_transform['label'] = np.where((subset_label_transform["label"] < 3) | (subset_label_transform["label"].isna()), 0, 1)
subset_label_transform

## Split the dataset in two subsets : the training and test sets

In [ ]:
from benchmark.utility import split_dataset
train_set, test_set = split_dataset(subset_label_transform, 0.2)

## Run tests

In [ ]:
import sys
import os

notebook_dir = os.path.dirname(os.path.abspath('__file__'))
src_dir = os.path.abspath(os.path.join(notebook_dir, '..', 'src'))

# Add 'src' directory to sys.path if it's not already there
if src_dir not in sys.path:
    sys.path.append(src_dir)

In [ ]:
from benchmark.utility import save_to_json
from benchmark.tests import n_shot_tests
from benchmark.train_eval_task import flair_f1_score

### N-shots

By default SetFit uses the oversampling strategy and the Cosine Similarity loss. For instance if we have 8 positive and 8 negative examples then we have:

|   | Y | Y | Y | Y | Y | Y | Y | Y | N | N | N | N | N | N | N | N |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Y | + | + | + | + | + | + | + | + | - | - | - | - | - | - | - | - |
| Y |   | + | + | + | + | + | + | + | - | - | - | - | - | - | - | - |
| Y |   |   | + | + | + | + | + | + | - | - | - | - | - | - | - | - |
| Y |   |   |   | + | + | + | + | + | - | - | - | - | - | - | - | - |
| Y |   |   |   |   | + | + | + | + | - | - | - | - | - | - | - | - |
| Y |   |   |   |   |   | + | + | + | - | - | - | - | - | - | - | - |
| Y |   |   |   |   |   |   | + | + | - | - | - | - | - | - | - | - |
| Y |   |   |   |   |   |   |   | + | - | - | - | - | - | - | - | - |
| N |   |   |   |   |   |   |   |   | + | + | + | + | + | + | + | + |
| N |   |   |   |   |   |   |   |   |   | + | + | + | + | + | + | + |
| N |   |   |   |   |   |   |   |   |   |   | + | + | + | + | + | + |
| N |   |   |   |   |   |   |   |   |   |   |   | + | + | + | + | + |
| N |   |   |   |   |   |   |   |   |   |   |   |   | + | + | + | + |
| N |   |   |   |   |   |   |   |   |   |   |   |   |   | + | + | + |
| N |   |   |   |   |   |   |   |   |   |   |   |   |   |   | + | + |
| N |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   | + |

- P = 2 * (8 + 7 + 6 + 5 + 4 + 3 + 2 + 1) 	= 72
- N = 8 * 8 = 64 -> + 8 duplications 		= 72
- Total = 72 + 72 = 144

In [ ]:
params = {
    "n_shot": [1, 2, 4, 6, 10],
    "n_iter": 5,
    "n_max_iter_per_shot": 5,
    "model": "flair-base",
    "loss": "CrossEntropyLoss"
}

results, train_times, eval_times = n_shot_tests(params, train_set, test_set, few_shot_model_f1_function=flair_f1_score)

save_to_json(results, train_times, eval_times, params,  r'../results/flair/n_shot')